## JWT Implementation



# What is JWT
JWT (JSon Web tokens) is a compact url safe means of representing claims bewtween two parties

# Parts of JWT

- Header

- Payload

- Signature

# Implementing JWT into Flask Project

# JWT Cookies
Cookies are pices of information taht are used to be sent to a browser and stored on the device it has been sent to

# Anatomy of Cookies

# Validating Cookies